# CIFAR 10

In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
from tqdm.notebook import trange

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import CIFAR10

device = 'cuda' if torch.cuda.is_available() else 'cpu'
%matplotlib widget
device

'cuda'

In [2]:
train_dataset = CIFAR10(
    root='./dataset',
    download=True,
    train=True,
    transform=ToTensor()
)
test_dataset = CIFAR10(
    root='./dataset',
    download=True,
    train=False,
    transform=ToTensor()
)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 128, 3, bias=False),
            nn.MaxPool2d(3),
            nn.ReLU(inplace=True),

#             nn.Conv2d(512, 512, 3),
#             nn.MaxPool2d(2),
#             nn.ReLU(inplace=True),

            nn.Conv2d(128, 64, 3),
            nn.MaxPool2d(3),
            nn.ReLU(inplace=True)
        )
        self.flat = nn.Flatten()
        self.hidden_ly = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
#             nn.Linear(512, 512),
#             nn.ReLU(inplace=True),
            nn.Linear(256, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flat(x)
        x = self.hidden_ly(x)
        return x

In [5]:
class BobNet(nn.Module):
    def __init__(self):
        super(BobNet, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 32, 5, padding=2, bias=False),
            nn.MaxPool2d(3),
            nn.ReLU(inplace=True),

#             nn.Conv2d(512, 512, 3),
#             nn.MaxPool2d(2),
#             nn.ReLU(inplace=True),

            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(3),
            nn.ReLU(inplace=True)
        )
        self.flat = nn.Flatten()
        self.hidden_ly = nn.Sequential(
            nn.Linear(288, 120),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flat(x)
        x = self.hidden_ly(x)
        return x

In [6]:
class BobNet(nn.Module):
    def __init__(self):
        super(BobNet, self).__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 32, 4, padding=0, bias=False),
            nn.MaxPool2d(3),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 32, 4, padding=0),
            nn.MaxPool2d(3),
            nn.ReLU(inplace=True)
        )
        self.flat = nn.Flatten()
        self.hidden_ly = nn.Sequential(
            nn.Linear(128, 120),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flat(x)
        x = self.hidden_ly(x)
        return x

In [7]:
def calc_acc(model, dt):
    X, y = dt
    X, y = X.to(device), y.to(device)  
    y_pred = model(X)
    y_hat = torch.argmax(y_pred, dim=1)
    
    wrng_p = torch.count_nonzero(y - y_hat) / len(y)
    return 1 - wrng_p

In [8]:
def train_step(model, dt, optim, loss_fn, test=False):
    X, y = dt
    X, y = X.to(device), y.to(device)
    y_hat = model(X)

    loss = loss_fn(y_hat, y)
    if not test:
        loss.backward()
        optim.step()
        
        for param in model.parameters():
            param.grad = None
            
    return loss.item(), calc_acc(model, dt)

In [9]:
def train_epoch(model, dtld, optim, loss_fn, test=False):
    epoch_losses = []
    epoch_acc = []

    for batch, dt in enumerate(dtld):
        bt_loss, bt_acc = train_step(model, dt, optim, loss_fn, test=test)
        epoch_losses.append(bt_loss)
        epoch_acc.append(bt_acc)
        
    return np.array(epoch_losses).mean(), torch.tensor(epoch_acc).mean()

In [10]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
batch_size = 32
lr=1e-4

model = Net().to(device)

tr_dtld = DataLoader(train_dataset, batch_size=batch_size)
ts_dtld = DataLoader(test_dataset, batch_size=batch_size)

optim = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

epochs = 100

tr_losses = []
ts_losses = []

torch.backends.cudnn.benchmark = True

for epoch in (t := trange(epochs)):
    ax.clear()
        
    tr_loss, tr_acc = train_epoch(model, tr_dtld, optim, loss_fn)
    ts_loss, ts_acc = train_epoch(model, ts_dtld, optim, loss_fn, test=True)
    
    tr_losses.append(tr_loss)
    ts_losses.append(ts_loss)
    
    t.write(f'Epoch {epoch}/{epochs} // TrAcc={tr_acc} // TsAcc={ts_acc}')
    t.set_description(f'Loss {ts_loss} / Acc {ts_acc}')
    
    ax.plot(range(epoch+1), tr_losses, label='Train losses')
    ax.plot(range(epoch+1), ts_losses, label='Test losses')
    ax.set_title(f'BS={batch_size} / lr={lr} \n\n Ts_acc={ts_acc}')
    ax.legend()
    fig.canvas.draw()

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0/100 // TrAcc=0.27471208572387695 // TsAcc=0.34375
Epoch 1/100 // TrAcc=0.37098127603530884 // TsAcc=0.38448482751846313
Epoch 2/100 // TrAcc=0.41478726267814636 // TsAcc=0.43320685625076294
Epoch 3/100 // TrAcc=0.44713690876960754 // TsAcc=0.45736822485923767
Epoch 4/100 // TrAcc=0.4693298041820526 // TsAcc=0.4715455174446106
Epoch 5/100 // TrAcc=0.4855446219444275 // TsAcc=0.48742014169692993
Epoch 6/100 // TrAcc=0.5065978765487671 // TsAcc=0.5095846652984619
Epoch 7/100 // TrAcc=0.5226127505302429 // TsAcc=0.5125798583030701
Epoch 8/100 // TrAcc=0.537447988986969 // TsAcc=0.5300519466400146
Epoch 9/100 // TrAcc=0.5515635013580322 // TsAcc=0.5384384989738464
Epoch 10/100 // TrAcc=0.5653191208839417 // TsAcc=0.5471246242523193
Epoch 11/100 // TrAcc=0.5729566812515259 // TsAcc=0.5537140369415283
Epoch 12/100 // TrAcc=0.5841130614280701 // TsAcc=0.5574081540107727
Epoch 13/100 // TrAcc=0.5928102731704712 // TsAcc=0.5654951930046082
Epoch 14/100 // TrAcc=0.6009677052497864 // TsAc